Script to output the necessary fields for the VWIND constraint for CMIP5.  This includes...

Ensemble mean past and future v for all scales and zonal wavenumbers greater than 3 in map form.
Southwest meridional wind indices (250E-265E, 25N-37N) for past and future, ensemble mean and first member.
Past stationary wave amplitude for k>3 averaged over 200E-300E, 20N-40N for ensemble mean and first member.

Output is put to ecpaper2020/DATASORT/VWIND/DATA/

In [14]:
import importlib
import pandas as pd
import xarray as xr
import numpy as np
import xesmf as xe
from numpy import nan
import sys
import warnings

from ecpaper_utils import readdata_utils as read
from ecpaper_utils import calendar_utils as cal
from ecpaper_utils import filter_utils as filt
from ecpaper_utils import averaging_utils as avg

importlib.reload(read)
importlib.reload(cal)
importlib.reload(filt)
importlib.reload(avg)

warnings.filterwarnings('ignore')



Set paths for CMIP5 models (historical and RCP8.5) and the pressure level to us (in Pa)

In [15]:
histpath="/project/cmip5/historical/Amon/"
rcp85path="/project/cmip5/rcp85/Amon/"
plevuse="30000"
pathout="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/DATA/"

Information ont he models being used is provided in cmip5csvinfo.csv.  This contains information on the models, number of members and whether there's any special order for the member numbers i.e., if they don't simply go from 1 to N.  Read in this info and set up the dates for each period.

In [16]:
cmip5models=pd.read_csv("../cmip5csvinfo.csv")

ybegp = 1979 ; monbegp = 1 ; yendp = 2014 ; monendp = 12 # dates for past period
ybegf = 2070 ; monbegf = 1 ; yendf = 2099 ; monendf = 12 # dates for Future period

# total number of months (used for checking)
nmonthsp = (yendp-ybegp-1)*12 + (12-monbegp+1) + monendp
nmonthsf = (yendf-ybegf-1)*12 + (12-monbegf+1) + monendf

# set up date names
datebegp=str(ybegp)+"-"+str(monbegp).zfill(2)
dateendp=str(yendp)+"-"+str(monendp).zfill(2)
datebegf=str(ybegf)+"-"+str(monbegf).zfill(2)
dateendf=str(yendf)+"-"+str(monendf).zfill(2)

In [17]:
models=cmip5models["Model"]
nmodels=models.size

# define 1 deg greid
grid_out = xr.Dataset({'lat': (['lat'], np.arange(-90,91,1.0)), 'lon': (['lon'], np.arange(0,360,1.0))})
nlon = grid_out["lon"].size ; nlat=grid_out["lat"].size

# define multi-model arrays
vpast_em = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vpast_em')
vpast_1mem = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vpast_1mem')
vfut_em = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vfut_em')
vfut_1mem = xr.DataArray(np.zeros([nmodels,nlat,nlon]), coords=[models, grid_out['lat'], grid_out['lon']], dims=['Model', 'lat', 'lon'], name='vfut_1mem')

# loop over models
for index, modname in models.iteritems():
    
    # initialize boolean for reusing weights
    wgtfile="/project/cas/islas/python/ecpaper2020/DATASORT/VWIND/tmp/wgt.nc"
    reusewgt=False
    
    # ---- the past period ----
    nmems=cmip5models.loc[index, "Nmempast"]
    
    for imem in range(1, nmems+1, 1): # loop over members
        memstr="r"+str(imem)+"i1p1"
        
        # check for a special order
        changeorder = cmip5models.loc[index, "specialorderpast"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing past for "+modname+" "+memstr+"...")
        histdir=histpath+"va/"+modname+"/"+memstr+"/"
        rcp85dir=rcp85path+"va/"+modname+"/"+memstr+"/"
        
        # read in meridional wind, concatenate historical to 2005, rcp85 to 2014
        vhist = read.read_1lev(histdir+"/*.nc", datebegp, "2005-12", plevuse)
        vrcp = read.read_1lev(rcp85dir+"/*.nc", "2006-01", dateendp, plevuse)
        v=xr.concat([vhist, vrcp], dim="time", join="override") 

        # check the array length.  If this fiales, try ending historical in 2005-11
        # and beginning rcp in 2005-12 to fix Hadley center
        if (v.time.size != nmonthsp):
            print("switching to the assumption that historical ends at 2005-11")
            vhist = read.read_1lev(histdir+"/*.nc", datebegp, "2005-11", plevuse)
            vrcp = read.read_1lev(rcp85dir+"/*.nc", "2005-12", dateendp, plevuse)
            v=xr.concat([vhist, vrcp], dim="time", join="override")
            
        # check again
        if (v.time.size != nmonthsp):
            print("something's wrong, nmonthsp="+str(nmonthsp)+" but v has size "+str(v.time.size))
            sys.exit("v past for "+modname+", "+memstr+" doesn't have the right size")
            
        # calculate DJF mean and interpolate
        vdjf = cal.season_mean(v, "va", season="DJF")
        regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        vdjfinterp = regridder(vdjf)
        
        if (imem == 1):
            vpast_1mem[index,:,:] = vdjfinterp
            reusewgt=True
            
        vpast_em[index,:,:] = vpast_em[index,:,:] + vdjfinterp[:,:]/float(nmems)
        
    # ---the future period-------
    
    nmems=cmip5models.loc[index, "Nmemfuture"]
    for imem in range(1,nmems+1,1):
        memstr="r"+str(imem)+"i1p1"
        changeorder=cmip5models.loc[index,"specialorderfuture"]
        if (type(changeorder) == str):
            changeordernp=np.array(changeorder.split(","))
            memstr="r"+str(changeordernp[imem-1])+"i1p1"
            
        print("Processing future for "+modname+" "+memstr)
        rcp85dir=rcp85path+"/va/"+modname+"/"+memstr+"/"
        v=read.read_1lev(rcp85dir+"/*.nc", datebegf, dateendf, plevuse)
        
        # check the right number of months are there
        if (v.time.size != nmonthsf):
            print("something's wrong, nmonthf="+str(nmonthsf)+" but v has size "+str(v.time.size))
            sys.exit("v future for "+modname+", "+memstr+" doesn't have the right size")
            
        vdjf = cal.season_mean(v, "va", season="DJF")
        regridder = xe.Regridder(vdjf, grid_out, 'bilinear', periodic=True, reuse_weights=reusewgt, filename=wgtfile)
        vdjfinterp = regridder(vdjf)
        
        if (imem == 1):
            vfut_1mem[index,:,:]=vdjfinterp
            reusewgt=True
            
        vfut_em[index,:,:] = vfut_em[index,:,:] + vdjfinterp[:,:]/float(nmems)
        
    # clean up regridding file
    !rm $wgtfile
    

Processing past for ACCESS1-0 r1i1p1...
Processing future for ACCESS1-0 r1i1p1
Processing past for ACCESS1-3 r1i1p1...
Processing future for ACCESS1-3 r1i1p1
Processing past for bcc-csm1-1 r1i1p1...
Processing future for bcc-csm1-1 r1i1p1
Processing past for bcc-csm1-1-m r1i1p1...
Processing future for bcc-csm1-1-m r1i1p1
Processing past for BNU-ESM r1i1p1...
Processing future for BNU-ESM r1i1p1
Processing past for CanESM2 r1i1p1...
Processing past for CanESM2 r2i1p1...
Processing past for CanESM2 r3i1p1...
Processing past for CanESM2 r4i1p1...
Processing past for CanESM2 r5i1p1...
Processing future for CanESM2 r1i1p1
Processing future for CanESM2 r2i1p1
Processing future for CanESM2 r3i1p1
Processing future for CanESM2 r4i1p1
Processing future for CanESM2 r5i1p1
Processing past for CCSM4 r1i1p1...
Processing past for CCSM4 r2i1p1...
Processing past for CCSM4 r3i1p1...
Processing past for CCSM4 r4i1p1...
Processing past for CCSM4 r5i1p1...
Processing past for CCSM4 r6i1p1...
Processing

Calculate indices and output to netcdf

In [18]:
# V_SW index
vpasteddy_em = vpast_em - vpast_em.mean("lon")
vfuteddy_em = vfut_em - vfut_em.mean("lon")
vpasteddy_1mem = vpast_1mem - vpast_1mem.mean("lon")
vfuteddy_1mem = vfut_1mem - vfut_1mem.mean("lon")

vpastsw_em = avg.cosweightlonlat(vpasteddy_em, 250, 265, 25, 37)
vpastsw_em = vpastsw_em.rename("vpastsw_em")
vpastsw_1mem = avg.cosweightlonlat(vpasteddy_1mem, 250, 265, 25, 37)
vpastsw_1mem = vpastsw_1mem.rename("vpastsw_1mem")
vfutsw_em = avg.cosweightlonlat(vfuteddy_em, 250, 265, 25, 37)
vfutsw_em = vfutsw_em.rename("vfutsw_em")
vfutsw_1mem = avg.cosweightlonlat(vfuteddy_1mem, 250, 265, 25, 37)
vfutsw_1mem = vfutsw_1mem.rename("vfutsw_1mem")

# stationary wave ampltiude index
vpast_em_k4to32 = filt.filterk(vpast_em, 4, 32)
vpast_em_k4to32 = vpast_em_k4to32.rename("vpast_em_k4to32")
vpastamp_em = np.sqrt(avg.cosweightlonlat(vpast_em_k4to32**2, 200., 300., 20., 40.))
vpastamp_em = vpastamp_em.rename("vpastamp_em")

vpast_1mem_k4to32 = filt.filterk(vpast_1mem, 4, 32)
vpastamp_1mem = np.sqrt(avg.cosweightlonlat(vpast_1mem_k4to32**2, 200., 300., 20., 40.))
vpastamp_1mem = vpastamp_1mem.rename("vpastamp_1mem")

# Future k>3
vfut_em_k4to32 = filt.filterk(vfut_em, 4, 32)
vfut_em_k4to32 = vfut_em_k4to32.rename("vfut_em_k4to32")

# output vwind indices
vwindindices = xr.merge([vpastsw_em, vfutsw_em, vpastsw_1mem, vfutsw_1mem, vpastamp_em, vpastamp_1mem])
vwindindices.to_netcdf(path=pathout+"cmip5vwindindices.nc")

# output vwind map data
vwindmapdata = xr.merge([vpast_em, vfut_em, vpast_em_k4to32, vfut_em_k4to32])
vwindmapdata.to_netcdf(path=pathout+"cmip5vwindmapdata.nc")
